In [1]:
import scrapy
from scrapy import Selector
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.keys import Keys
from webdriver_manager.chrome import ChromeDriverManager
from logzero import logfile, logger
from random import randint

In [2]:
from dotenv import load_dotenv
import os
import json
import pandas as pd
import numpy as np
from datetime import datetime
import twitchAPI.types as types

datetime_inicio_str='2022-01-01 00:00:00'
datetime_fim_str='2022-03-31 23:59:59'
datetime_inicio = datetime.strptime(datetime_inicio_str, '%Y-%m-%d %H:%M:%S')
datetime_fim = datetime.strptime(datetime_fim_str, '%Y-%m-%d %H:%M:%S')

#Carrega variaveis de ambiente para parametrização
load_dotenv('vars.env')

from twitchAPI.twitch import Twitch

#Autenticacao
twitch = Twitch(os.environ.get('cli_id'), os.environ.get('cli_token'))

# Get Clips

### Obter ultimos 100 clips de videos no periodo

In [ ]:
df_bruto = pd.read_csv('DF_STREAM_PT_NOME_IDS_wteam.csv', sep=';')

In [ ]:
df_clips = pd.DataFrame({'id': pd.Series(dtype='str'),
                              'url': pd.Series(dtype='str'),
                              'user_login': pd.Series(dtype='str'),
                              'embed_url': pd.Series(dtype='str'),
                              'broadcaster_id': pd.Series(dtype='str'),
                              'broadcaster_name': pd.Series(dtype='str'),
                              'creator_id': pd.Series(dtype='str'),
                              'creator_name': pd.Series(dtype='str'),
                              'video_id': pd.Series(dtype='str'),
                              'game_id': pd.Series(dtype='str'),
                              'language': pd.Series(dtype='str'),
                              'title': pd.Series(dtype='str'),
                              'view_count': pd.Series(dtype='int'),
                              'created_at': pd.Series(dtype='str'),
                              'thumbnail_url': pd.Series(dtype='str'),
                              'duration': pd.Series(dtype='float'),
                              })
df_clips.to_csv(f'clips_jan_mar2022.csv', mode='w', sep=";", header=True, index=False)
for index, row in df_bruto.iterrows():
    try:
        resultado_consulta = twitch.get_clips(broadcaster_id=row['broadcaster_id'], first=100, started_at=datetime_inicio, ended_at=datetime_fim)
        data_resultado_consulta = pd.DataFrame(resultado_consulta['data'])
        data_resultado_consulta.to_csv('clips_jan_mar2022.csv', mode='a', sep=";", header=False, index=False)
    except:
        print('user nao tem clip', row['broadcaster_id'])

In [21]:
df_clips = pd.read_csv(f'clips_jan_mar2022.csv', sep=";", encoding='iso-8859-1')
df_clips.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 222347 entries, 0 to 222346
Data columns (total 16 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   id                222347 non-null  object 
 1   url               222347 non-null  object 
 2   user_login        222347 non-null  object 
 3   embed_url         222347 non-null  int64  
 4   broadcaster_id    222347 non-null  object 
 5   broadcaster_name  222347 non-null  int64  
 6   creator_id        222347 non-null  object 
 7   creator_name      89164 non-null   float64
 8   video_id          221314 non-null  float64
 9   game_id           222234 non-null  object 
 10  language          222330 non-null  object 
 11  title             222347 non-null  int64  
 12  view_count        222347 non-null  object 
 13  created_at        222347 non-null  object 
 14  thumbnail_url     222347 non-null  float64
 15  duration          0 non-null       float64
dtypes: float64(4), int64

# Get videos
    tipos de video:
    -ALL
    -UPLOAD
    -ARCHIVE
    -HIGHLIGHT
    -UNKNOWN
- Atualmente NAO é possivel via api associar as categorias de um video. não tem vinculado a informação de capitulos
- e para achar os marcadores, precisa de autenticacao

### Obter categorias/capitulos do video

In [ ]:
df_split = np.array_split(df_bruto, 11)
for n in range(0,11):
    df = df_split[n]
    df = df.loc[(df.total_followers >= 30)] #há 13.997  streamns com menos menos 60% de 50 seguidores(30) que é o minimo do objetivo para se tornar afiliado da twitch
    service = Service(executable_path=ChromeDriverManager().install())
    driver = webdriver.Chrome(service=service)
    driver.get('chrome://settings/clearBrowserData') #limpa o browser
    #Criação do arquivo
    schema_arquivo = pd.DataFrame({
                        'id': pd.Series(dtype='str'),
                        'stream_id': pd.Series(dtype='str'),
                        'user_id': pd.Series(dtype='str'),
                        'user_login': pd.Series(dtype='str'),
                        'user_name': pd.Series(dtype='str'),
                        'title': pd.Series(dtype='str'),
                        'description': pd.Series(dtype='str'),
                        'created_at': pd.Series(dtype='str'),
                        'published_at': pd.Series(dtype='str'),
                        'url': pd.Series(dtype='str'),
                        'thumbnail_url': pd.Series(dtype='str'),
                        'viewable': pd.Series(dtype='str'),
                        'view_count': pd.Series(dtype='int'),
                        'language': pd.Series(dtype='str'),
                        'type': pd.Series(dtype='str'),
                        'duration': pd.Series(dtype='str'),
                        'muted_segments': pd.Series(dtype='str'),
                        'game_id': pd.Series(dtype='str'),
                        'game_name': pd.Series(dtype='str'),
                        'game_duration': pd.Series(dtype='str')
    })
    schema_arquivo.to_csv(f'chapters_pt{n}.csv', mode='w', sep=";", header=True, index=False)
    #Iteração do broadcaster_id
    for index_id, row_id in df.iterrows():
        #Busca ultimas 100 lives
        resultado_consulta = twitch.get_videos(user_id=row_id['broadcaster_id'], first=100, period=types.TimePeriod.ALL, sort=types.SortMethod.TIME)
        try:
            data_resultado_consulta = pd.DataFrame(resultado_consulta['data'])
            #filtrar lives dentro do periodo de estudo
            data_resultado_consulta['created_at'] = pd.to_datetime(data_resultado_consulta['created_at'])
            data_resultado_consulta = data_resultado_consulta[(data_resultado_consulta['created_at'] >= datetime_inicio_str)&(data_resultado_consulta['created_at'] <= datetime_fim_str)]
            data_resultado_consulta['game_id'] = None
            data_resultado_consulta['game_name'] = None
            data_resultado_consulta['game_duration'] = None
            #Iteração das urls do broadcaster_id
            for index, row in data_resultado_consulta.iterrows():
                tentativa = 0
                driver.get(f"{row['url']}") #Acessa url do video
                driver.implicitly_wait(6)
                try:
                    driver.find_elements(By.XPATH, "//*[contains(text(), 'Começar a assistir')]")[0].click() #liberar conteudo video
                except:
                    pass
                try:
                    driver.find_elements(By.XPATH, '//*[@id="root"]/div/div[2]/div[1]/main/div[2]/div[3]/div/div/div[2]/div/div[2]/div/div/div/div/div[6]/div/div[2]/div[1]/div[2]/div[2]/button')[0].click() #capitulos
                    driver.implicitly_wait(3)
                    lista_elem_capitulos = driver.find_elements(By.XPATH, "//div[@class='Layout-sc-nxg1ff-0 jAEOcJ']")
                    #Iteração dos capitulos das urls do broadcast_id
                    for elem in range(0,len(lista_elem_capitulos)):
                        nome_capitulo = driver.find_elements(By.XPATH, "//div[@class='Layout-sc-nxg1ff-0 media-row__info-text']")[elem].text 
                        game_id = twitch.get_games(names=[nome_capitulo])['data'][0]['id']
                        duracao = driver.find_elements(By.XPATH, "//div[@class='Layout-sc-nxg1ff-0 media-row__info-description']")[elem].text 
                        row['game_id'] = game_id
                        row['game_name'] = nome_capitulo
                        row['game_duration'] = duracao
                        pd.DataFrame([row.to_dict()]).to_csv(f'chapters_pt{n}.csv', mode='a', sep=";", header=False, index=False)            
                        # print(nome_capitulo, game_id, duracao) #debug
                except:
                    nome_capitulo = driver.find_elements(By.XPATH, '//*[@id="root"]/div/div[2]/div[1]/main/div[2]/div[3]/div/div/div[1]/div[1]/div[2]/div/div[1]/div/div[1]/div[1]/div[2]/div/div/a/p')[0].text
                    game_id = twitch.get_games(names=[nome_capitulo])['data'][0]['id']
                    duracao = None
                    row['game_id'] = game_id
                    row['game_name'] = nome_capitulo
                    row['game_duration'] = duracao
                    # print(nome_capitulo, game_id, duracao) #debug
                    pd.DataFrame([row.to_dict()]).to_csv(f'chapters_pt{n}.csv', mode='a', sep=";", header=False, index=False) 

        except:
            nome_capitulo = pd.DataFrame([row_id.to_dict()]).to_csv('streammers_sem_video.csv', mode='a', sep=";", header=False, index=False)

In [16]:
df_videos = pd.read_csv('chapters_pt0.csv', sep=';', encoding='iso-8859-1')
for n in range(1,11):
    df_video_temp = pd.read_csv(f'chapters_pt{n}.csv', sep=';', encoding='iso-8859-1')
    df_videos = pd.concat([df_videos, df_video_temp], axis=0)
df_videos.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 37899 entries, 0 to 4776
Data columns (total 20 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   id              37899 non-null  int64  
 1   stream_id       37899 non-null  int64  
 2   user_id         37899 non-null  int64  
 3   user_login      37899 non-null  object 
 4   user_name       37899 non-null  object 
 5   title           37842 non-null  object 
 6   description     1330 non-null   object 
 7   created_at      37899 non-null  object 
 8   published_at    37898 non-null  object 
 9   url             37899 non-null  object 
 10  thumbnail_url   37878 non-null  object 
 11  viewable        37899 non-null  object 
 12  view_count      37899 non-null  object 
 13  language        37899 non-null  object 
 14  type            37899 non-null  object 
 15  duration        37886 non-null  object 
 16  muted_segments  13 non-null     float64
 17  game_id         37899 non-null  

In [17]:
df_videos.to_csv(f'videos_with_chapters.csv', mode='w', sep=";", header=True, index=False, encoding='iso-8859-1')